In [ ]:
import requests
import time

all_found = set()
with open('./oss_fuzz_corpus_urls.txt', 'r') as f:
    for url in f.readlines():
        all_found.add(url.strip())
        
def verify_and_add(project, fuzzer):
    # do not download the corpus
    url1 = f'https://storage.googleapis.com/{project}-backup.clusterfuzz-external.appspot.com/corpus/libFuzzer/{project}_{fuzzer}/public.zip'
    url2 = f'https://storage.googleapis.com/{project}-backup.clusterfuzz-external.appspot.com/corpus/libFuzzer/{fuzzer}/public.zip'
    if url1 in all_found or url2 in all_found:
        print(f'Already verified {project}:{fuzzer}')
        return

    r = requests.head(url1)
    time.sleep(0.1)
    if r.status_code == 200:
        print(f'Verified {project}:{project}_{fuzzer}')
        all_found.add(url1)
    else:
        r = requests.head(url2)
        time.sleep(0.1)
        if r.status_code == 200:
            print(f'Verified {project}:{fuzzer}')
            all_found.add(url2)
        else:
            print(f'Could not verify {project}:{fuzzer}')


In [ ]:
# # from oss-fuzz repo
# # NOTE: first clone oss-fuzz

# # find all projects in ./oss-fuzz/projects
# import os
# import time

# for project in os.listdir('./oss-fuzz/projects'):
#     if not os.path.isdir(f'./oss-fuzz/projects/{project}'):
#         continue
    
#     # recursive search for fuzzers
#     blacklist = ['.dict', '.diff', '.h', '.proto', '.options', '.patch', '.zip', '.xml', 'make_fuzzers']
#     maybe_fuzzers = [f.split('.')[0] 
#                      for _, _, files in os.walk(f'./oss-fuzz/projects/{project}') 
#                      for f in files 
#                      if 'fuzz' in f.lower()
#                      and not any([b in f for b in blacklist])]

#     for fuzzer in maybe_fuzzers:
#         verify_and_add(project, fuzzer)

In [ ]:
# # from fuzzbench repo
# # NOTE: first clone fuzzbench

# import yaml
# from glob import glob

# # find all .yaml files in ./fuzzbench/benchmarks
# yaml_files = glob('./fuzzbench/benchmarks/**/*.yaml', recursive=True)
# for yaml_file in yaml_files:
#     with open(yaml_file, 'r') as f:
#         data = yaml.safe_load(f)
#         project = data['project']
#         fuzzer = data['fuzz_target']
#         verify_and_add(project, fuzzer)


In [ ]:
# # from oss-fuzz-vulns repo + oss-fuzz issues

# import backoff
# import re
# import time

# from glob import glob

# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options

# # setup headless chrome
# chrome_options = Options()
# chrome_options.add_argument('--headless')
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')
# driver = webdriver.Chrome(options=chrome_options)

# ISSUE_REGEX = re.compile(r'https://bugs.chromium.org/p/oss-fuzz/issues/detail\?id=\d+')
# PROJECT_REGEX = re.compile(r'[Pp]roject[\s\w*]*:\s*(\w+)')
# FUZZER_REGEX = re.compile(r'[Ff]uzz [\w*\s]*[Tt]arget[\s\w*]*:\s*(\w+)')

# # fetch page and wait for javascript to load
# def fetch_html(url):
#     driver.get(url)
#     # wait for both project and fuzzer to load
#     for _ in range(5):
#         if PROJECT_REGEX.search(driver.page_source) and FUZZER_REGEX.search(driver.page_source):
#             return driver.page_source
#         time.sleep(1)
#     else:
#         raise Exception('Timed out waiting for page to load')

# # find all issue urls in ./oss-fuzz-vulns
# all_issue_urls = set()
# for report in glob('./oss-fuzz-vulns/**/*.yaml', recursive=True):
#     with open(report, 'r') as f:
#         urls = ISSUE_REGEX.findall(f.read())
#         for url in urls:
#             all_issue_urls.add(url)

# print(f'Found {len(all_issue_urls)} issue urls')

# for url in all_issue_urls:
#     try:
#         print(f'Fetching {url}')
#         html = fetch_html(url)
#     except KeyboardInterrupt:
#         raise
#     except:
#         print(f"Failed to fetch {url}")
#         continue
#     # find and print Project and Fuzz Target (cannot contain newline or spaces)
#     project = PROJECT_REGEX.search(html).group(1)
#     fuzzer = FUZZER_REGEX.search(html).group(1)
#     verify_and_add(project, fuzzer)


In [ ]:
# from oss-fuzz built targets

# 1) clone oss-fuzz
# 2) build all fuzzers for all projects, for example: 
#    (but use a job manager and redirect stdout/stderr)
#    cd oss-fuzz/infra; find ../projects -mindepth 1 -maxdepth 1 -type d | parallel -j10 'python3 helper.py build_fuzzers {/}'

# find all executable files in oss-fuzz/build/out
import os
from glob import glob
for project in os.listdir('./oss-fuzz/build/out'):
    for maybe_fuzzer in glob(f'./oss-fuzz/build/out/{project}/*'):
        # must be executable, contain 'fuzz' in the name, and not have any extensions
        if os.access(maybe_fuzzer, os.X_OK) and 'fuzz' in os.path.basename(maybe_fuzzer).lower() and '.' not in os.path.basename(maybe_fuzzer):
            fuzzer = os.path.basename(maybe_fuzzer)
            verify_and_add(project, fuzzer)

In [5]:
# make sure to keep all urls we've already found
with open('./oss_fuzz_corpus_urls.txt', 'r') as f:
    for url in f.readlines():
        all_found.add(url.strip())
with open('./oss_fuzz_corpus_urls.txt', 'w') as f:
    for url in all_found:
        f.write(url + '\n')